In [2]:
import numpy as np
import pandas as pd


FILTER CRITERIA:

-- Phase 2, 3, 4

-- Only include single-arm trial if there’s a suitable outcome measure (e.g. fMRI)

-- Include combined modality trials (e.g. NeuroNext, oxytocin + behavioral intervention)

-- Sort DataFrame by drugs (after I tease out that column / make a Placebo column) and compare to existing drug database


In [70]:
df = pd.read_csv('./datasets/20211015_drug_trials_unfiltered.csv')

In [71]:
len(df)

2012

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2012 entries, 0 to 2011
Columns: 159 entries, Unnamed: 0 to WhyStopped
dtypes: float64(3), int64(10), object(146)
memory usage: 2.4+ MB


In [73]:
df.head()

,Unnamed: 0,index,Rank,Acronym,ArmGroupDescription,ArmGroupInterventionName,ArmGroupLabel,ArmGroupType,BriefSummary,BriefTitle,...,Rank.7,StdAge,StudyFirstPostDate,StudyFirstPostDateType,StudyFirstSubmitDate,StudyFirstSubmitQCDate,StudyPopulation,StudyType,VersionHolder,WhyStopped
0,0,0,1,Gene&autism,"Probands with Autism Spectrum Disorder, (N=700...",Genetic: DNA from subjects will be stored in t...,Autism Spectrum Disorder|Control without Autis...,NaN,"The main objective of the study is to define, ...",Study of the Genetic Factors Involved in Autis...,...,1,Child|Adult|Older Adult,"January 27, 2021",Actual,"January 20, 2021","January 22, 2021",For all patients included in the study,Observational,"October 15, 2021",NaN
1,1,1,2,NaN,"For all patients included in the study, core a...",NaN,Autism Spectrum Disorder|controls,NaN,"The main objective of the study is to define, ...",Susceptibility Genes in Autism Spectrum Disorders,...,2,Child|Adult|Older Adult,"December 11, 2015",Estimate,"September 26, 2013","December 9, 2015","For all patients included in the study, .",Observational,"October 15, 2021",NaN
2,2,2,3,NaN,Children who meet criteria based on expert cli...,Device: Marcus Autism Center Investigational D...,Autism Spectrum Disorder|Non-autism Spectrum D...,NaN,"This is an outpatient, multicenter, prospectiv...",A Multi-site Comparison of Social Visual Engag...,...,3,Child,"March 19, 2018",Actual,"March 12, 2018","March 16, 2018",Subjects will be recruited on the basis of con...,Observational,"October 15, 2021",NaN
3,3,3,4,NaN,Children affected by autism spectrum disorders...,Behavioral: Children with autism spectrum diso...,Subjects with autism spectrum disorders|Health...,NaN,"To date, it is well documented that the gut mi...",Dietetic Pattern in Children With Autism Spect...,...,4,Child,"January 22, 2021",Actual,"January 18, 2021","January 18, 2021",200 subjects of both sexes diagnosed with Auti...,Observational,"October 15, 2021",NaN
4,4,4,5,Q10ASD,If body weight is up to 20 kg:||Vitamin E 30 m...,Dietary Supplement: Vitamin E|Dietary Suppleme...,PMS Placebo|ASD Placebo|PMS Active compound|AS...,Active Comparator|Active Comparator|Experiment...,"This double-blind, cross-over, randomized, con...",Q10 Ubiquinol in Autism Spectrum Disorder and ...,...,5,Child|Adult,"March 18, 2020",Actual,"March 9, 2020","March 17, 2020",NaN,Interventional,"October 15, 2021",NaN


In [74]:
df_phase = df[df['Phase'].notna()]

In [75]:
len(df_phase)

1533

DROP TRIALS THAT LIST 'NOT APPLICABLE' FOR PHASE. SEE: https://clinicaltrials.gov/ct2/about-studies/glossary#:~:text=Not%20Applicable%20is%20used%20to,the%20safety%20of%20a%20drug.

In [76]:
df_phase['Phase'].head()

4     Not Applicable
5            Phase 3
8     Not Applicable
9     Not Applicable
13           Phase 2
Name: Phase, dtype: object

In [77]:
df_phase = df_phase[~df_phase['Phase'].str.contains("Applicab")]

In [78]:
len(df_phase)

715

In [79]:
df_phase['Phase'].head()

5     Phase 3
13    Phase 2
22    Phase 1
23    Phase 2
28    Phase 2
Name: Phase, dtype: object

DROP PHASE 1 TRIALS.

In [80]:
df_phase = df_phase[~df_phase['Phase'].str.contains("1")]

In [81]:
len(df_phase)

519

In [82]:
df_phase['InterventionType']

5                                               Drug|Drug
13                                         Drug|Drug|Drug
23                                                  Other
28                                                   Drug
38                                             Drug|Other
                              ...                        
1996                                                 Drug
2000             Drug|Drug|Drug|Drug|Drug|Other|Drug|Drug
2002    Drug|Biological|Drug|Drug|Drug|Drug|Drug|Drug|...
2007                                       Drug|Drug|Drug
2010                                      Drug|Behavioral
Name: InterventionType, Length: 519, dtype: object

KEEP ONLY ROWS THAT CONTAIN 'DRUG' WITHIN INTERVENTION TYPE.

In [83]:
df_phase_drugs = df_phase[df_phase['InterventionType'].str.contains("Drug")]

In [84]:
len(df_phase_drugs)

443

Expand the ArmGroupInterventionName column, to view the various drugs tested. Then, EXPORT .CSV FILE, TEMPORARILY. MORE FILTERING MAY BE REQUIRED.

In [85]:
df_phase_drugs_unique = df_phase_drugs['ArmGroupInterventionName'].str.split('|', expand=True)

In [86]:
df_phase_drugs_unique.to_csv('datasets/20211015_drug_trials_filtered_phase_drugs_only.csv')

In [89]:
df_phase_drugs.to_csv('datasets/20211015_drug_trials_filtered_phase_drugs.csv')

In [90]:
df_phase_drugs = df_phase_drugs[~df_phase_drugs['EligibilityCriteria'].str.contains('Politte')]

In [91]:
len(df_phase_drugs)

443

In [106]:
targets = ['Autis', 'autis', 'Rett', 'Angelman', 'uberous', 'Phelan', '15q', 'Fragile X', 'Asperger', 'Williams', 'Pervasive Developmental Disorder']

In [107]:
extra_filter_mask = df_phase_drugs['ConditionBrowseLeafAsFound'].str.contains('Autis|autis|Rett|Angelman|uberous|Phelan|15q|Fragile X|Asperger|Williams|Pervasive Developmental Disorder', regex=True, na=False)
extra_filter_mask

5        True
13       True
28       True
38       True
42       True
        ...  
1996    False
2000    False
2002    False
2007    False
2010    False
Name: ConditionBrowseLeafAsFound, Length: 443, dtype: bool

In [109]:
s = pd.Series(extra_filter_mask)
s

5        True
13       True
28       True
38       True
42       True
        ...  
1996    False
2000    False
2002    False
2007    False
2010    False
Name: ConditionBrowseLeafAsFound, Length: 443, dtype: bool

ALSO LOOK AT ROWS LOST TO THIS FILTERING, EXPORT LOST ROWS TO CSV TO STUDY.

In [110]:
df_phase_drugs_extra_filter = df_phase_drugs[s.values]

In [112]:
len(df_phase_drugs_extra_filter)

312

In [117]:
def check_for_placebo(row):
    if 'Placebo' in row['ArmGroupInterventionName']:
        return 'Yes'
    else:
        return 'No'

In [123]:
df_phase_drugs_extra_filter['Placebo'] = df_phase_drugs_extra_filter['ArmGroupInterventionName'].apply(lambda x: 'Yes' if 'Placebo' in str(x) else 'No')

C:\Users\nsmcc\AppData\Local\Temp/ipykernel_14572/2400386331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phase_drugs_extra_filter['Placebo'] = df_phase_drugs_extra_filter['ArmGroupInterventionName'].apply(lambda x: 'Yes' if 'Placebo' in str(x) else 'No')


Change data types of columns.

In [126]:
df_phase_drugs_extra_filter.head()

,Unnamed: 0,index,Rank,Acronym,ArmGroupDescription,ArmGroupInterventionName,ArmGroupLabel,ArmGroupType,BriefSummary,BriefTitle,...,StdAge,StudyFirstPostDate,StudyFirstPostDateType,StudyFirstSubmitDate,StudyFirstSubmitQCDate,StudyPopulation,StudyType,VersionHolder,WhyStopped,Placebo
5,5,5,6,Lisphem,Li+ is an FDA (NDA: 016834) and ANSM (AMM 3400...,Drug: Lithium Carbonate|Drug: Placebo,Lithium|Placebo,Experimental|Placebo Comparator,There is currently no treatment for the body s...,Effect of Lithium in Patients With Autism Spec...,...,Child|Adult,"November 10, 2020",Actual,"November 5, 2020","November 9, 2020",NaN,Interventional,"October 15, 2021",NaN,Yes
13,13,13,14,NaN,intranasal ketamine and dexmedetomidine was tr...,Drug: ketamine and dexmedetomidine|Drug: ketam...,ketamine and dexmedetomidine group|ketamine gr...,Experimental|Experimental|Placebo Comparator,Autism spectrum disorder (ASD) is a life-long ...,Intranasal Ketamine With Dexmedetomidine for t...,...,Child,"February 15, 2018",Actual,"January 28, 2018","February 9, 2018",NaN,Interventional,"October 15, 2021",NaN,No
28,28,28,29,Autism,"15 mg/kg oral solution three times per day, ma...",Drug: EPI-743,EPI-743,Experimental,The investigators hypothesize that EPI-743 may...,An Exploratory Open Label Study of EPI-743 (Vi...,...,Child,"August 27, 2014",Estimate,"August 21, 2014","August 25, 2014",NaN,Interventional,"October 15, 2021",This study was halted prior to enrolling.,No
38,38,38,39,NaN,Participants in the placebo arm will receive p...,Drug: Memantine|Other: Placebo,Memantine|Placebo|Control Group,Experimental|Placebo Comparator|No Intervention,"This study is a 12-week, randomized-controlled...",Behavioral and Neural Response to Memantine in...,...,Child,"October 30, 2013",Estimate,"October 24, 2013","October 29, 2013",NaN,Interventional,"October 15, 2021",NaN,Yes
42,42,42,43,NaN,Patients with Phelan-McDermid syndrome receive...,Drug: Recombinant human Growth hormone,Phelan-McDermid syndrome,Experimental,Phelan McDermid syndrome (PMS) is a rare genet...,Growth Hormone Treatment in Children With Phel...,...,Child,"July 1, 2019",Actual,"June 27, 2019","June 27, 2019",NaN,Interventional,"October 15, 2021",NaN,No


Now I want to create a new column, called Drugs, that lists out the various Drugs that are not listed as Placebo. 

In [127]:
df_phase_drugs_extra_filter.to_csv('datasets/20211015_drug_trials_filtered_phase_drugs_extra.csv')